# geetools chart module

This `geetools` module relies on `pygal` library, so the returned charts are instances of `pygal.chart`. See options at 
[pygal site][1]

I made a JavaScript 'equivalent': https://code.earthengine.google.com/b2922b860b85c1120250794fb82dfda8

* **Author**: Rodrigo E. Principe
* **email**: fitoprincipe82@gmail.com
* **GitHub**: github.com/fitoprincipe
* **Repository GeeTools**: github.com/gee-community/gee_tools

  [1]: http://www.pygal.org/en/latest/documentation/index.html  

## Install

You can install `geetools` using `pip`:

    pip install geetools

If you can't or don't want to use pip, you can install it directly in the notebook:

In [1]:
# Uncomment and run this cell if you want to install geetools directly
# on the notebook

# import sys
# !{sys.executable} -m pip install --upgrade geebap

## Import packages

In [2]:
import ee
from geetools import chart

## Make some test sites

In [3]:
test_site = ee.Geometry.Point([-71, -42])

In [4]:
test_feat = ee.Feature(test_site, {'name': 'test feature'})

In [5]:
test_featcol = ee.FeatureCollection([
    test_feat, 
    test_feat.buffer(100).set('name', 'buffer 100'),
    test_feat.buffer(1000).set('name', 'buffer 1000')
])

## Time Series

In [6]:
years = ee.List([2015, 2016, 2017, 2018])

In [7]:
col = ee.ImageCollection('COPERNICUS/S2').filterBounds(test_site)

In [8]:
def make_time_series(year):
    ''' make a time series from year's list '''
    eefilter = ee.Filter.calendarRange(year, field='year')
    filtered = col.filter(eefilter)
    return filtered.mean().set('system:time_start', ee.Date.fromYMD(year, 1, 1).millis())

In [9]:
time_series = ee.ImageCollection(years.map(make_time_series))

## Chart: *series*

In [10]:
chart_ts = chart.Image.series(**{
    'imageCollection': time_series, 
    'region': test_site,
    'scale': 10,
    'bands': ['B1', 'B2', 'B3'],
    # 'xProperty': 'B4', # You can use a band too!
    'labels': ['band B1', 'B2 band', 'this is B3']
})

In [11]:
chart_ts.render_widget(width='50%')

SFRNTCh2YWx1ZT11JzxlbWJlZCBzcmM9ZGF0YTppbWFnZS9zdmcreG1sO2NoYXJzZXQ9dXRmLTg7YmFzZTY0LFBEOTRiV3dnZG1WeWMybHZiajBuTVM0d0p5QmxibU52WkdsdVp6MG5kWFJtTFTigKY=


## Chart: *seriesByRegion*

In [12]:
chart_ts_region = chart.Image.seriesByRegion(**{
    'imageCollection': time_series,
    'reducer': ee.Reducer.median(),
    'regions': test_featcol,
    'scale': 10,
    'band': 'B11',
    'seriesProperty': 'name'
})

In [13]:
chart_ts_region.render_widget(height=500)

SFRNTCh2YWx1ZT11JzxlbWJlZCBzcmM9ZGF0YTppbWFnZS9zdmcreG1sO2NoYXJzZXQ9dXRmLTg7YmFzZTY0LFBEOTRiV3dnZG1WeWMybHZiajBuTVM0d0p5QmxibU52WkdsdVp6MG5kWFJtTFTigKY=
